# Predicting Beer Ratings from User Reviews: Capstone 1 Milestone Report
*****

### Define the Problem
To some beer is simply an alcoholic beverage that one can indulge to relax or become active in social environments, but to some it is an entire market of styles, flavors and art that is crafted and perfected. Through the increase in breweries and global connection through technologies there are so many options for beer enthusiasts to discover and it can becoming highly overwhelming due to this over saturation of products. For this reason, consumers often rely on ratings and reviews submitted by others to determine if a new product is worth trying. However, the accuracy of these reviews may not necessarily match the product itself. When you look at a product you typically see a rating system giving it an average score, but what if the review text itself doesn’t match the rating given by the user? This can decrease sales for the seller and misguide the consumer in search for a new beer. Is it possible to build a machine-learning model using Natural Language Processing and sentiment analysis to predict ratings based off the reviews? I will try to do just that in the efforts to create a more accurate review platform for beers that could help both the sellers and enthusiasts.


The process required to brew a new beer can take anywhere from one to two months. In that period not only is a lot of time invested into perfecting the new beverages, but costs as well. Reviews and ratings can help a beer brewer make informed decisions on what type of beers to experiment with and which ones to stay away from in any particular area or region. Likewise, reviews can also help beer enthusiasts discover new beverages that fall along their taste palates. On the other hand, incomplete or “troll” reviews can misguide both parties as well. To determine how effective these ratings really are, I suggest using various features of beer reviews to predict beer ratings. 

***
### Potential Clients
My main client would be beer brewers at all levels. This information could be valuable for big corporations such as Budweiser all the way down to your local brewery that is about to start up. The machine learning algorithm could be used to sift through their consumer reviews to provide the most accurate and consistent ratings to make these informed decisions on beer to increase sales and customer satisfaction. It can also help shed light at the development phase by showing what styles are popular to see if a new combination could become a big seller.

***
### Data Set Description
The dataset contains over 1.5 million reviews of various beers from two websites: BeerAdvocate.com. This data not only includes user reviews, product category and alcohol by volume(ABV), but sensory aspects as well such as taste, look, smell and overall ratings. For this project I will train and test models to predict beer ratings and beer style based off the user reviews that were left.

These reviews were made available by Julian Mcauley, a UCSD Computer Science professor, from a collection period of January 1998 to November 2011. This dataset was accessed with permission. Here are some key specs of the dataset itself.
+ Number of users: 33,387
+ Number of items: 66,051
+ Number of reviews: 1,586,259
+ Rating Scales: Appearance, Aroma, Palate, Taste, Overall
    
To tackle the issue of size, I took a subset sample of 99,999 reviews to train and test our models before applying it to the rest of the dataset.

***
### Data Wrangling 
There were a few problems I ran into when cleaning the dataset. Each problem is listed below with a solution I came up for it.

__Problem:__ The different ratings left behind by users were on different scales. Aroma and Taste were on a scale of 10, while Appearance and Palate were on a scale of 5. The overall rating was also on an unknown scale.

__Solution:__ To remedy this, I scaled down "Aroma" and "Taste" down to a 5-point scale to match the other. I also combined the four sensory ratings and averaged them to get an "Overall" rating. The code for this is below. 

In [ ]:
beer['aroma'] = round((4 * beer['aroma'] / 9 + (5/9))).astype(int)
beer['taste'] = round((4 * beer['taste'] / 9 + (5/9))).astype(int)
beer['overall'] = (beer['appearance'] + beer['aroma'] + beer['palate'] + beer['taste']) / 4

__Problem:__ The text reviews have a bunch of miscellaneous information in them. This includes spelling errors, punctuation, stop words, contractions, capitalization and whitespace. Also, having different tenses of a word is not necessary. For example the words plays, played, playing all come from the base word play. I only want the base word.

__Solution:__ To clean the text, I created a bunch of functions to lowercase the text, expand contractions, remove whitespace and remove stop words. In addition I used to common practices in NLP, stemming and lemmatization to derive the base of each word. Although lemmatization took more time, I found it more effective and chose too keep the lemmatized text reviews over the stemmed ones.

In [ ]:
def simple_stemmer(text):
    """Apply the PorterStemmer() onto the given text to extract only the base of the word"""
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


def lemmatize_text(text):
    """Use spacy to lemmatize the text"""
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

__Problem:__ Non-English Text Reviews. Some of the text reviews were not in English. This is an issue because the non-English words throw the model off. 

__Solution:__ Using lang detect I searched through the data and dropped any rows of text reviews that were not identified as English. Sometimes it was unable to detect the language. For these I used a try and except clause to ignore the reviews that a language could not be identified.

In [ ]:
# In the length of the beer dataframe
for i in range(len(beer2)):
    try:
        if detect(str(beer2['clean_text_lem'][i])) != 'en':   # If the language detected is not english, drop the row
            beer2.drop(i, inplace=True)
    except:     # Pass the loop if no language can be identified
        pass

To wrap up data wrangling I did the following to the dataset:
+ Took a subset of the entire data collected
+ Dropped unnecessary columns: time, beerId, brewerId
+ Fixed spelling errors
+ Broke down text reviews in preparation for sentiment analysis
+ Added a new feature: word_count
+ Extracted five beer styles to examine along with the entire dataset: Amber Ale, Belgian Ale, Brown Ale, IPA, Stout
+ Started with dataset size of 99,999. After cleaning and removing data that it was reduced to 81048.

More can be seen within the Data Wrangling code on my Github

***
### Exploratory Data Analysis

__1. Beer Styles__

Examining beer styles I noticed that there are 79 different beer styles within this portion of the dataset. However, a majority of them do not have lots of reviews. To tackle this issue I did two things. 
1. Remove any beer styles with less than 1000 reviews. This shrunk the style of beers down from 79 to 27 as can be seen in the two plots below. 

<tr>
    <td> <img src="plots/beerstyle79.png" alt="All Beer Styles" style="width: 450px;"/> </td>
    <td> <img src="plots/beerstyle27.png" alt="Beer Styles w/ 1000+ reviews" style="width: 425px;"/> </td>
</tr>

2. After narrowing down the styles, I also chose to examine closely 5 distinct beer styles based on the number of reviews.
    + India Pale Ales (IPA)
    + Stout
    + Amber Ale
    + Brown Ale
    + Belgian Ale
    
    


***
__2. Rating Scales__

There are 5 different rating scales for each review written: Appearance, Taste, Aroma, Palate(Feel), Overall

The first four are categorical. Users gave a score of 1, 2, 3, 4, or 5.                                                 
The overall rating is an average of Appearance, Taste, Aroma and Palate(Feel).

Later on, I will try to predict each rating based on the text reviews. Here is some more information about each scale.

1. Appearance
    + Describes the beer's color, clarity, head retention and lacing. 
    + Head retention is how long a beer can keep a layer of foam at the top of the beer after being poured into a cup.
    + Lacing is the residue left from the foam at the top of the beer.
    
    
2. Taste
    + Describes the flavor characteristics of the beer. This includes the malts, the hops, the overall finish of the beer, the       aftertaste, and any other byproducts left behind from fermentation. 
    
    
3. Aroma
    + Describes the aromatics of the beer. This includes the malts, hops and yeast, which are essential to making beer.
    + Similar to wine, smelling the beer enhances the experience of drinking one.
    
    
4. Palate (Feel)
    + Describes the feeling of the beer within the mouth. This includes the carbonation, warmth, creaminess and overall body of       the beer.
    
    
5. Overall
    + Describes the overall impression of the beer 
    + An average of the other 4 rating scales

From the plots and data observed it appears that the Appearance scale it is a little bit different.
+ Stouts have the highest average rating 
+ Brown Ales and IPAs have similar average ratings. Slightly leaning towards the 4 value they are very close to the middle.
+ Amber Ales follow the trend of the entire dataset also near the middle value but leaning slightly towards the 3 value.
+ Belgian Ales consistently had a lower average rating amongst all scales include Appearance.

The plot for the appearance ratings is below.

<img src="plots/appearance_ratings.png" alt="Appearance Ratings" />

The Overall, Aroma, Feel and Taste scale follow similar patterns. 
+ Stout and IPA have the highest average ratings leaning closer to the value of 4. Both have similar ratings
+ Brown Ales resemble the distribution of the entire dataset combined together.
+ Belgian Ales and Amber Ales have the lowest average ratings leaning closer to the value of 3.

The plot for the overall ratings is shown below.

<img src="plots/overall_ratings.png" alt="overall ratings"/>

***
__3. Engineered Features__

__WORD COUNT__

The first feature I made was the length of each text review by determining the word count. Naturally this data was skewed to the right as there were some extremely excited or utterly disappointed users who had to express their opinions in lengthy reviews. To check for normality, I transformed the count using log and lo and behold it looked much better. 

<img src="plots/word_count.png" alt="Word Prob"/>
***
<img src="plots/word_count_log.png" alt="Log Word Prob"/>

***
__LOW and HIGH Reviews__

Next, the majority of reviews fell between 3 and 4 on the rating scale of 1-5. To compare reviews further I created a new feature to create a low vs high scale. It was two category feature for the overall rating scale. Reviews less than 3.5 were considered "low" and reviews greater than or equal to 3.5 were considered "high". Below I've displayed the plots shows the difference in log_word_counts for the entire dataset and the five styles of beer.

<img src="plots/high_low_words.png" alt="word_frequency_1"/>

Looking at the chart you can see that across all styles and the dataset as whole, higher rated reviews(>3.5) have more words per review than low rated reviews. Using an independent two sample t-test for unequal variances I determined that this was statistically significant.

***
__4. Word Frequencies__
The most important feature of this dataset will be the text within each text review. To further understand this I created a frequency word plot as seen below. 

Examining the top 25 words I found that describing words such as 'sweet', 'light' and 'white' were used frequently. I also saw words such as 'aroma', 'flavor' and 'taste' which correspond to the different rating scales. The most common word was 'head' in reference to the foam at the top of the beer when it is first poured. This may need to be removed later on when running models. In addition positive words such as 'good' and 'nice' could be seen.

One key point to watch out for when creating a Naive Bayes model will be that the word 'not' appeared quite a bit. This may change the polarity of a review so the n-grams may play an important factor in predicting rating scales. 

<img src="plots/wordfreq.png" alt="word_frequency_1"/>

***
### Summary

I found a few patterns when looking through the data. 
1. Beer reviews are dominated by 3 or 4 star reviews. There are some 5 and 2 star reviews and little to no 1 star reviews. These patterns remain similar across all rating categories. 
2. The average amongst all rating scales lies roughly around the 3.5 star rating mark. This makes sense because most ratings fell between 3 or 4 star reviews. 
3. When splitting the reviews into low and high based on the 3.5 star rating average mark, higher rated reviews have higher word counts
3. There are a limited number of quantitative variables for this dataset so the main focus will be on the text reviews themselves and the models applied to it. 
4. Additional words may need to be removed from the text reviews to remove noise.

When creating the predictor model it will be important to look at n-grams to see if putting certain words together will help avoid incorrect predictions.

***
## Sources
Lipton, Zachary & Vikram, Sharad & McAuley, Julian. (2015). Capturing Meaning in Product Reviews with Character-Level Generative Text Models.
https://www.researchgate.net/publication/283761921_Capturing_Meaning_in_Product_Reviews_with_Character-Level_Generative_Text_Models